In [122]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import hvplot.pandas
from pathlib import Path
from numpy import mean
from numpy import std
from numpy import cov
from scipy.stats import pearsonr
import requests
import urllib.request
import json
import pprint
from dotenv import load_dotenv

%matplotlib inline

In [123]:
load_dotenv()
access_token = os.getenv("CQ_API_KEY")
headers = { 'Authorization': 'Bearer ' + access_token }



In [124]:
URL = "https://api.cryptoquant.com/v1/btc/exchange-flows/netflow?exchange=all_exchange&window=day&from=20150101&limit=100"
response = requests.get(URL, headers=headers)

netflow_data = response.json()
netflow_df = pd.DataFrame(data=netflow_data['result']['data'])
netflow_df['date'] = pd.to_datetime(netflow_df['date'])
netflow_df['netflow_total'] = netflow_df['netflow_total'].astype(str).astype('float64')
netflow_df = netflow_df.dropna()








In [125]:
URL2 = "https://api.cryptoquant.com/v1/btc/market-data/price-usd?window=day&from=20150101&limit=100"
response2 = requests.get(URL2, headers=headers)

price_data = response2.json()

price_df = pd.DataFrame(data=price_data['result']['data'])
price_df = price_df.drop(['price_usd_high','price_usd_low','price_usd_open'], axis = 1)
price_df['date'] = pd.to_datetime(price_df['date'])
price_df['price_usd_close'] = price_df['price_usd_close'].astype(str).astype('float64')
price_df = price_df.dropna()




In [126]:
covariance = cov(netflow_df['netflow_total'],price_df['price_usd_close'])
corr, _ = pearsonr(netflow_df['netflow_total'], price_df['price_usd_close'])
print('Pearsons correlation: %.3f' % corr)


Pearsons correlation: 0.097


In [127]:
def pearsons_coefficient(series1,series2):
    
  #Returns pearsons coefficient (int) between 2 dataframes columns.

       # Parameters: 
            ## series1 (object): First series to compare
            ## series2 (object): Second series to compare

    covariance = cov(series1,series2)
    corr, _ = pearsonr(series1,series2)
    print('Pearsons correlation: %.3f' % corr)
    return corr



In [128]:
test_df = pearsons_coefficient(netflow_df['netflow_total'],price_df['price_usd_close'])

Pearsons correlation: 0.097


In [144]:
def get_onchain_data(ticker,fromdate,limit):
    '''
    Returns on-chain data (Netflow)n in new df
        Parameters:
            ticker (string) = Ticker for Desired Coin
            fromdate (string) = Starting date of period desired
            limit (int) = number of data points     
    '''
    #Initialize dotenv function, pull API key, build URL
    headers = {'Authorization': 'Bearer ' + access_token}
    URL = f'https://api.cryptoquant.com/v1/{ticker}/exchange-flows/netflow?exchange=all_exchange&window=day&from={fromdate}&limit={limit}'

    # Initialize Dictionary
    response = requests.get(URL, headers=headers)
    
    netflow_data = response.json()

    #Transform to Dataframe and adjust data type
    netflow_df = pd.DataFrame(data=netflow_data['result']['data'])
    netflow_df['date'] = pd.to_datetime(netflow_df['date'])
    netflow_df['netflow_total'] = netflow_df['netflow_total'].astype(str).astype('float64')
    netflow_df = netflow_df.dropna()
    return netflow_df


In [130]:
#test1_df = get_onchain_data("btc","20190101","100")

#print(test1_df)

In [131]:
URL3 = "https://api.cryptoquant.com/v1/btc/market-data/capitalization?window=day&from=20191001&limit=10000"
response3 = requests.get(URL3, headers=headers)

mvrv_data = response3.json()

mvrv_data = pd.DataFrame(data=mvrv_data['result']['data'])
mvrv_data['mvrv_score'] = (mvrv_data['market_cap']/mvrv_data['realized_cap'])
mvrv_data['date'] = pd.to_datetime(mvrv_data['date'])
mvrv_data = mvrv_data.sort_values(by=['date'],ascending=False)
mvrv_data = mvrv_data.drop(['average_cap', 'delta_cap', 'market_cap','realized_cap','thermo_cap'], axis = 1)


mvrv_data_plot= mvrv_data.hvplot(x='date',figsize=(100,100), rot=90)
mvrv_data_plot
#mvrv_data.head(5)


:Curve   [date]   (mvrv_score)

In [136]:
def get_mvrv_data(ticker,fromdate,limit):
    '''
    Returns marketcap data in new df
        Parameters:
            ticker (string) = Ticker for Desired Coin
            fromdate (string) = Starting date of period desired
            limit (int) = number of data points     
    '''
    #Initialize dotenv function, pull API key, build URL
    headers = {'Authorization': 'Bearer ' + access_token}
    URL3 = f'https://api.cryptoquant.com/v1/{ticker}/market-data/capitalization?window=day&from={fromdate}&limit={limit}'
            
    # Initialize Dictionary
    response3 = requests.get(URL3, headers=headers)
    
    mvrv_data = response3.json()

    mvrv_data = pd.DataFrame(data=mvrv_data['result']['data'])
    mvrv_data['mvrv_score'] = (mvrv_data['market_cap']/mvrv_data['realized_cap'])
    mvrv_data['date'] = pd.to_datetime(mvrv_data['date'])
    mvrv_data = mvrv_data.sort_values(by=['date'],ascending=False)
    mvrv_data = mvrv_data.drop(['average_cap', 'delta_cap', 'market_cap','realized_cap','thermo_cap'], axis = 1)

    return mvrv_data



In [141]:
#test1_df = get_mvrv_data("btc","20190101","100000")
    

In [142]:
def get_plots(input_df):
    '''
    Returns plot of inputed data frame
        Parameters:
        input_df: consumes the output of df builder functions above
    '''
    
    data_plot= input_df.hvplot(x='date',figsize=(100,100), rot=90)
    return data_plot

In [143]:
mvrv_plot = get_plots(mvrv_data)
mvrv_plot

:Curve   [date]   (mvrv_score)

In [146]:
netflow_plot = get_plots(netflow_df)
netflow_plot

:Curve   [date]   (netflow_total)

In [147]:
price_df_plot = get_plots(price_df)
price_df_plot

:Curve   [date]   (price_usd_close)